In [5]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
sample="When Alex started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."
sample2="Kunarasa was born in 1941 in Jaffna and died on 28 February 2016"

In [8]:
doc = nlp(sample)

In [9]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Alex 5 9 PERSON
Google 50 56 ORG
2007 60 64 DATE


In [1]:
! pip install ktrain

^C


In [2]:
import ktrain
from ktrain import text

Using TensorFlow backend.


using Keras version: 2.2.4


## Read the input json file

In [37]:
import json
with open("documents_train.json","r") as read_file:
    data = json.load(read_file)

In [66]:
myData=[]
for document in data:
    text=document['text']
    H0=document['H0']
    H1=document['H1']
    i=0
    sentence=""
    categ="None"
    entities=[]
    while(i<len(text)):
        if(len(sentence)==0):
            start_index=0
        else:
            start_index=len(sentence)
        if(H0[i]==1):
            entities.append([start_index,start_index+len(text[i]),"H0"])
            if(categ!="H0"):
                categ='H0'
        if(H1[i]==1 and H0[i]==0):
            entities.append([start_index,start_index+len(text[i]),"H1"])
            if(categ!="H1" and categ!="H0"):
                categ='H1'
        sentence=sentence+" "+text[i]
        if('.' in text[i]):
            myData.append([sentence,categ,entities])
            sentence=""
            categ="None"
            entities=[]
        i+=1
            
    

In [75]:
myData

[[' Isaac David Abella (June 20, 1934 – October 23, 2016) was a Professor of Physics at The University of Chicago.',
  'H0',
  [[0, 5, 'H0'],
   [6, 11, 'H0'],
   [12, 18, 'H0'],
   [25, 28, 'H1'],
   [29, 33, 'H1'],
   [34, 35, 'H1'],
   [36, 43, 'H1'],
   [44, 47, 'H1'],
   [48, 53, 'H1'],
   [54, 57, 'H1'],
   [60, 69, 'H1'],
   [70, 72, 'H1'],
   [73, 80, 'H1'],
   [81, 83, 'H1'],
   [84, 87, 'H1'],
   [88, 98, 'H1'],
   [99, 101, 'H1'],
   [102, 110, 'H1']]],
 [' He specialized in laser physics, quantum optics, and spectroscopy.',
  'NA',
  []],
 [' Isaac was the cousin of Irving Abella.',
  'H0',
  [[0, 5, 'H0'], [24, 30, 'H0'], [31, 38, 'H0']]],
 [' Isaac Abella was born on June 20, 1934 in Toronto, Ontario.',
  'H0',
  [[0, 5, 'H0'],
   [6, 12, 'H0'],
   [25, 29, 'H1'],
   [30, 33, 'H1'],
   [34, 38, 'H1'],
   [42, 50, 'H1'],
   [51, 59, 'H1']]],
 [' Abella received his Bachelor of Arts degree (1957) from the University of Toronto, Master of Arts (1959) degree, and Ph.',
  'H0'

In [68]:
import pandas as pd
my_df = pd.DataFrame(myData)
my_df.columns=['Sentence', 'Category', 'Spacy']

In [69]:
my_df.to_csv('sent_classifier_train.csv', index=False)

## Building the Sentence Classification model

We will be using Kashgiri library to build our model and will be evaluating its performance against multiple algorithms

In [76]:
import kashgari

In [77]:
from kashgari.corpus import SMP2018ECDTCorpus

In [106]:
#Importing libraries to create data to be used for training
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
set(stopwords.words('english'))
import string
table = str.maketrans(dict.fromkeys(string.punctuation)) #Used to remove the punctuations

In [138]:
data = pd.read_csv("sent_classifier_train.csv", index_col=False)
X= data.Sentence
Y=data.Category


In [169]:
stop_words = set(stopwords.words('english')) 
X_model=[]
for item in X:
    sentence=item.lower()
    #Remove punctuations
    new_sentence=sentence.translate(table)
    #Tokenize the transformed sentence
    word_tokens=word_tokenize(new_sentence)
    sample=[w for w in word_tokens if not w in stop_words]
    if(len(sample)>0):
        X_model.append(sample)

In [176]:
X_model

[['isaac',
  'david',
  'abella',
  'june',
  '20',
  '1934',
  '–',
  'october',
  '23',
  '2016',
  'professor',
  'physics',
  'university',
  'chicago'],
 ['specialized', 'laser', 'physics', 'quantum', 'optics', 'spectroscopy'],
 ['isaac', 'cousin', 'irving', 'abella'],
 ['isaac', 'abella', 'born', 'june', '20', '1934', 'toronto', 'ontario'],
 ['abella',
  'received',
  'bachelor',
  'arts',
  'degree',
  '1957',
  'university',
  'toronto',
  'master',
  'arts',
  '1959',
  'degree',
  'ph'],
 ['1963', 'physics', 'columbia', 'university', 'new', 'york'],
 ['studied', 'charles', 'h'],
 ['townes', 'involved', 'early', 'research', 'work', 'laser', 'development'],
 ['notably',
  'abellas',
  'thesis',
  'townes',
  'among',
  'earliest',
  'work',
  'twophoton',
  'absorption'],
 ['married',
  'mary',
  'ann',
  'abella',
  'professor',
  'art',
  'chicago',
  'state',
  'university'],
 ['son', 'benjamin', 'daughter', 'sarah'],
 ['abella',
  'known',
  'work',
  'laser',
  'coherent',

In [175]:
df_X = pd.DataFrame({'Tokenized_Examples': X_model})

AttributeError: 'list' object has no attribute 'values'

In [212]:
#Split data into training, testing, and  in 60:20:20 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, Y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [213]:
import kashgari
from kashgari.tasks.classification import BiLSTM_Model
import logging
logging.basicConfig(level='DEBUG')

In [214]:
X_train_list=X_train.Tokenized_Examples.tolist()
X_val_list=X_val.Tokenized_Examples.tolist()
y_train=y_train.tolist()
y_val=y_val.tolist()

In [215]:
import kashgari
from kashgari.tasks.classification import BiGRU_Model
from kashgari.embeddings import BERTEmbedding

import logging
logging.basicConfig(level='DEBUG')

bert_embed = BERTEmbedding('BERT',
                           task=kashgari.CLASSIFICATION,
                           sequence_length=100)
model = BiGRU_Model(bert_embed)
model.fit(X_train_list, y_train, X_val_list, y_val)

W1124 15:01:41.669465  3928 bert_embedding.py:126] seq_len: 100


Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

Epoch 1/5
15/15 [==============================] - 397s 26s/step - loss: 1.7213 - acc: 0.4967 - val_loss: 0.8126 - val_acc: 0.6769
Epoch 2/5
15/15 [==============================] - 381s 25s/step - loss: 0.6537 - acc: 0.7193 - val_loss: 0.6377 - val_acc: 0.7424
Epoch 3/5
15/15 [==============================] - 381s 25s/step - loss: 0.5270 - acc: 0.7961 - val_loss: 0.6406 - val_acc: 0.7642
Epoch 4/5
15/15 [==============================] - 382s 25s/step - loss: 0.4330 - acc: 0.8509 - val_loss: 0.6036 - val_acc: 0.7991
Epoch 5/5
15/15 [==============================] - 401s 27s/step - loss: 0.3435 - acc: 0.8827 - val_loss: 0.5761 - val_acc: 0.7773


In [216]:
# Evaluate the model
X_test_list=X_test.Tokenized_Examples.tolist()
y_test=y_test.tolist()
model.evaluate(X_test_list, y_test)

              precision    recall  f1-score   support

          H0     0.8741    0.8613    0.8676       137
          H1     0.7143    0.6429    0.6767        70
        None     0.7159    0.7975    0.7545        79

    accuracy                         0.7902       286
   macro avg     0.7681    0.7672    0.7663       286
weighted avg     0.7913    0.7902    0.7897       286

